In [38]:
%pip install -q sql av pillow pandas sqlalchemy ipython-sql pymysql roboflow git+https://github.com/THU-MIG/yolov10.git supervision huggingface_hub bottleneck==1.3.6 numexpr==2.8.4

In [9]:
import pandas as pd
import pymysql
from pymysql.err import IntegrityError, OperationalError
import os
from PIL import Image
import numpy as np
import supervision as sv
import cv2
import av
from PIL import Image
import shutil as sh
import re
from roboflow import Roboflow
import cv2
from ultralytics import YOLOv10
import wget


In [3]:
def annotate_image(input_img, label_annotator, bounding_box_annotator, model_pred, verbose = False):
    cont_img = np.ascontiguousarray(input_img, dtype=np.uint8)
    results = model_pred(cont_img, conf=0.10, verbose = verbose)[0]
    detections = sv.Detections.from_ultralytics(results)
    annotated_image = bounding_box_annotator.annotate(
        scene=input_img, detections=detections)
    annotated_image = label_annotator.annotate(
        scene=annotated_image, detections=detections)
    return(annotated_image)

def annotate_video(in_path, out_path = '.', model_file = 'best.pt', im_width = 416, im_height = 416):
    bba = sv.BoundingBoxAnnotator()
    la = sv.LabelAnnotator()
    model = YOLOv10(model_file)

    container = av.open(in_path)
    stream_vid = container.streams.video[0]
    fname = in_path.rsplit('/', 1)[-1]
    per_index = fname.index('.')
    out_path = f'{out_path}/{fname[:per_index]}_annotated.mp4'
    outp = av.open(out_path, 'w')
    codec_name = stream_vid.codec_context.name
    fps = stream_vid.codec_context.rate
    output_stream = outp.add_stream(codec_name, str(fps))
    output_stream.width = im_width
    output_stream.height = im_height
    output_stream.pix_fmt = stream_vid.codec_context.pix_fmt
    for index, frame in enumerate(container.decode(stream_vid)):
        pil_img = frame.to_image()
        np_img = np.array(pil_img)
        np_img_resize = cv2.resize(np_img, (im_width, im_height))
        np_rot = np_img_resize[:, :, ::-1]
        small_pil_img = Image.fromarray(np_rot)
        np_image_2 = np.array(small_pil_img)
        an_mg = annotate_image(np_image_2, label_annotator = la, bounding_box_annotator=bba, model_pred = model, verbose = False)
        frame_out = av.VideoFrame.from_ndarray(an_mg, format='bgr24')
        pkt = output_stream.encode(frame_out)
        outp.mux(pkt)
    container.close()
    outp.close()

    print(f'Annotated video has been saved as {out_path}')
    return True

In [4]:
db_main = 'test_4'
cur_name = 'ab'


In [5]:
sql_conn = pymysql.connect(
    user="root",
    password="dbuserdbuser",
    host="localhost",
    port=3306,
    database=db_main,
    cursorclass=pymysql.cursors.DictCursor,
    autocommit=True)

cur = sql_conn.cursor()
res = cur.execute("SELECT 1")
res = cur.fetchall()
res

[{'1': 1}]

In [6]:
def add_user(name, password):
    '''
    Returns: 1 if query is successful, 0 if not
    '''

    try:
        res = cur.execute(f"INSERT INTO people (Username, Password, Time_Created) VALUES ('{name}', '{password}', CURRENT_TIMESTAMP );")
    except IntegrityError:
        print(f"Integrity Error Violated: Duplicate User with {name}")
        res = 0
    return res
add_user(cur_name, '123')

Integrity Error Violated: Duplicate User with ab


0

In [13]:
def get_ext(fpath):
    fname = fpath.rsplit('/', 1)[-1]
    per_index = fname.index('.')
    ext = fname[per_index + 1:].lower()
    return ext

def get_id_fname(f_out, fpath, id):
    fname = fpath.rsplit('/', 1)[-1]
    per_index = fname.index('.')
    ext = fname[per_index + 1:].lower()
    return f"{f_out}/{fname[:per_index]}-{id}.{ext}"

def get_REPLACE_ID(column_id = 'Raw_File_ID', table='raw_files', column_rep='Filepath'):
    cur.execute(f"""SELECT {column_id} from {table} where {column_rep} = 'REPLACE'""")
    id = cur.fetchall()[-1][column_id]
    return id

def get_match(pattern, string):
    matches = re.search(pattern, string)
    if matches:

        print(matches.group(1))
        return(matches.group(1))
    else:
        raise ValueError(f'Improper string exported, couldn\'t find {pattern} in the given text')

def delete_folder(name, base = "."):
    fpath = os.path.join(base, name)
    if os.path.exists(fpath):
        for item in os.listdir(fpath):
            new_path = os.path.join(fpath, item)
            if os.path.isfile(new_path):
                os.remove(new_path)
            elif os.path.isdir(new_path):
                delete_folder(item, fpath)
        os.rmdir(fpath)

def make_folder(name, base = ".", overwrite = True):
    fpath = os.path.join(base, name)
    delete_folder(name, base) if overwrite else None
    if os.path.exists(fpath):
        print("Filepath already exists")
    else:
        os.mkdir(fpath)

In [10]:


def add_photo(name, fpath, notes = '', f_out = './Files/Image_raw'):
    '''
    Returns: Index of added photo if successful, 0 if not
    '''

    im = Image.open(fpath)
    width = im.size[0]
    height = im.size[1]
    fsize = os.stat(fpath).st_size
    ext = get_ext(fpath)

    cur.execute(f"INSERT INTO raw_files (Username, Filepath, Size, Type, Extension, Notes, Width, Height, Time_Uploaded) VALUES ('{name}', 'REPLACE', {fsize}, 'Image', '{ext}', '{notes}', {width}, {height}, CURRENT_TIMESTAMP);")
    id = get_REPLACE_ID()
    f_id_name = get_id_fname(f_out, fpath, id)
    im.save(f_id_name)
    cur.execute(f"UPDATE raw_files SET Filepath = '{f_id_name}' WHERE Raw_File_ID = {id};")
    return id





id_raw_photo = add_photo(cur_name, fpath = 'C:/Users/micha/OneDrive/Desktop/Shellfish_project_2024/Jupyter_local_code/dummy.jpg', notes = 'after significant changes')

In [11]:
import cv2
def add_video(name, fpath, notes = '', f_out = './Files/Video_raw'):
    '''
    Returns: Index of added video if successful, 0 if not
    '''
    cap = cv2.VideoCapture(fpath)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fsize = os.stat(fpath).st_size
    ext = get_ext(fpath)
    
    cur.execute(f"INSERT INTO raw_files (Username, Filepath, Size, Type, Extension, Notes, Width, Height, Time_Uploaded) VALUES ('{name}', 'REPLACE', {fsize}, 'Video', '{ext}', '{notes}', {width}, {height}, CURRENT_TIMESTAMP);")
    id = get_REPLACE_ID()
    f_id_name = get_id_fname(f_out, fpath, id)
    sh.copyfile(fpath, f_id_name)

    cur.execute(f"UPDATE raw_files SET Filepath = '{f_id_name}' WHERE Raw_File_ID = {id};")

    fps = cap.get(cv2.CAP_PROP_FPS)
    color_order = 'RGB' # FIXME - cant figure out how to extract from cv2 object

    cur.execute(f"INSERT INTO videos (Raw_File_ID, FPS, Color_Order) VALUES ('{id}', '{fps}', '{color_order}');")

    return id

fpath_vid = "C:/Users/micha/OneDrive/Desktop/Shellfish_project_2024/GOPR1077.MP4"
id_raw_video = add_video(cur_name, fpath_vid, notes='without dummy files')

In [33]:

        

def add_roboflow(name, export_string, f_out = './Files/Roboflow', f_weights = "./Files/Weights"):
    
    '''
    Returns: Index of added roboflow if successful, 0 if not
    '''
    pattern_api = r'api_key\s*=\s*"([^"]+)"'
    api_key_lab = get_match(pattern_api, export_string)

    pattern_workspace = r'rf\.workspace\("([^"]+)"\)'
    workspace_lab = get_match(pattern_workspace, export_string)

    pattern_project = r'project\s*=\s*rf\.workspace\("[^"]+"\)\.project\("([^"]+)"\)'
    project_lab = get_match(pattern_project, export_string)

    pattern_version = r'\bproject\.version\((\d+)\)'
    version_lab = get_match(pattern_version, export_string)

    pattern_download = r'\bdownload\("([^"]+)"\)'
    download_lab = get_match(pattern_download, export_string)

    folder_roboflow = f"{f_out}/{workspace_lab}_{project_lab}_{version_lab}_{download_lab}"

    # if not os.path.exists(folder_roboflow):
    rf = Roboflow(api_key = api_key_lab)
    project = rf.workspace(workspace_lab).project(project_lab)
    version = project.version(version_lab)
    version.download(download_lab, location = folder_roboflow)
    
    fpath = f'{folder_roboflow}\\data.yaml'

    with open(fpath) as f1:
        lines = f1.readlines()
    with open(fpath, 'w') as f2:
        f2.writelines(lines[:-4])
        f2.write("test: ../test/images\ntrain: ../train/images\nval: ../valid/images\n\n")

    if not os.path.exists(f_weights):
        make_folder(f_weights)
        prefix = "https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10"
        versions = ['n', 's', 'm', 'b', 'x', 'l']
        suffix = ".pt"
        for ver in versions:
            web_path = prefix + ver + suffix
            wget.download(web_path, out = f_weights)
    cur.execute(f"INSERT INTO roboflow (Api_Key, Workspace, Project, Version, Download, Dataset_Location, Username, Timestamp) VALUES ('REPLACE', '{workspace_lab}', '{project_lab}', '{version_lab}', '{download_lab}', '{folder_roboflow}', '{name}', CURRENT_TIMESTAMP);")
    
    id = get_REPLACE_ID('Roboflow_ID', 'roboflow', 'Api_Key')
    
    cur.execute(f"UPDATE roboflow SET Api_Key = '{api_key_lab}' WHERE Roboflow_ID = {id};")



    return id




    
id_rob = add_roboflow(cur_name, \
"""
rf = Roboflow(api_key="Rvk9Clz4ote58KBvCfU3")
project = rf.workspace("curvy-oysters").project("oysters-cjt7n")
dataset = project.version(19).download("yolov8")

""" )




Rvk9Clz4ote58KBvCfU3
curvy-oysters
oysters-cjt7n
19
yolov8
loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.1.34, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to ./Files/Roboflow/curvy-oysters_oysters-cjt7n_19_yolov8 in yolov8:: 100%|██████████| 5668/5668 [00:01<00:00, 2869.03it/s]


In [41]:
def add_model(name, roboflow_ID, f_out = "./Files/Model", tmp_path = "./Files/tmp", weights_path = "./Files/Weights/yolov10n.pt"):
    make_folder(tmp_path)
    print(os.getcwd())
    # print(pt)
    cur.execute(f"SELECT * FROM roboflow WHERE Roboflow_ID = {roboflow_ID}")
    res = cur.fetchall()[-1]
    print(weights_path)
    pt = os.path.join(os.getcwd(), res['Dataset_Location'][2:], 'data.yaml')
    pt_rep = pt.replace('\\', '/')
    data_path = './Files/Weights/yolov10n.pt'
    !yolo task=detect mode=train epochs=10 batch=32 plots=False model={data_path} data={pt_rep}
    delete_folder(tmp_path)

add_model(cur_name, id_rob)
    

c:\Users\micha\OneDrive\Desktop\Shellfish_project_2024\sql
./Files/Weights/yolov10n.pt
c:\Users\micha\OneDrive\Desktop\Shellfish_project_2024\sql\Files/Roboflow/curvy-oysters_oysters-cjt7n_19_yolov8\data.yaml
./Files/Weights/yolov10n.pt


In [37]:
print(id_raw_photo)
print(id_raw_video)
print(id_rob)

6
7
17
